In [ ]:
import os
import gradio as gr
import google.generativeai as genai
from dotenv import load_dotenv
import pandas as pd

# -------------------------------
# Setup & Configuration
# -------------------------------

# Load environment variables from .env file
load_dotenv()

# Retrieve the API key for Gemini
api_key = os.getenv("Gemini_API_Key")
if not api_key:
    raise ValueError("API key not found. Ensure 'Gemini_API_Key' is correctly set in the .env file.")

genai.configure(api_key=api_key)

# Load the wine dataset
nlp_file_path = "../Resources/nlp_3.1_final.csv"
wine_df = pd.read_csv(nlp_file_path)

# -------------------------------
# Define Options for the UI
# -------------------------------

# Wine Color Options
wine_colors = ["Red", "White", "Rosé", "Orange", "Sparkling"]

# Define valid flavor and aroma options for each wine color
wine_attributes = {
    "Red": {
        "flavors": ["Berries", "Stone Fruits", "Dried Fruits", "Spices", "Woody", "Chocolate & Coffee", "Oaky", "Earthy"],
        "aromas": ["Fruity Red Wine - Red Berries", "Fruity Red Wine - Stone Fruits", "Maturation in Oak Barrel - Woods", "Maturation in Oak Barrel - Spices"]
    },
    "White": {
        "flavors": ["Citrus", "Tropical Fruits", "Floral", "Vegetal", "Mineral", "Caramel & Vanilla", "Sweet"],
        "aromas": ["Fruity White Wine - Citrus", "Fruity White Wine - Pome Fruits", "Floral - White Flowers"]
    },
    "Rosé": {
        "flavors": ["Berries", "Stone Fruits", "Floral", "Savory", "Spices"],
        "aromas": ["Fruity Red Wine - Red Berries", "Floral - Colored Flowers", "Vegetal - Fresh Herbs"]
    },
    "Orange": {
        "flavors": ["Dried Fruits", "Vegetal", "Nutty", "Toasty", "Smoky"],
        "aromas": ["Fruity White Wine - Botrytized", "Maturation in Oak Barrel - Nuts", "Maturation in Oak Barrel - Toasted"]
    },
    "Sparkling": {
        "flavors": ["Citrus", "Tropical Fruits", "Sweet", "Toasty", "Nutty"],
        "aromas": ["Fruity White Wine - Citrus", "Floral - White Flowers", "Maturation in Oak Barrel - Toasted"]
    }
}

# Wine body options
body_options = ["Light-bodied", "Medium-bodied", "Full-bodied"]

# Price Selection
price_options = ["$1 - $9.99", "$10 - $99.99", "$100 - $999.99", "$1000+"]

# -------------------------------
# Core Functions
# -------------------------------

def get_wine_recommendation(wine_color, flavor_profile, aroma, sweetness, body, price_range):
    filtered_wines = wine_df[
        wine_df["color"].apply(lambda x: wine_color in x) &
        wine_df["flavors"].apply(lambda x: flavor_profile in x) &
        wine_df["aromas"].apply(lambda x: aroma in x)
    ]
    
    if not filtered_wines.empty:
        recommended_wine = filtered_wines.sample(1).iloc[0]
        wine_name = recommended_wine['Name']
        wine_recommendation = f"""
**Recommended Wine:**  
**{wine_name}**

**Attributes:**  
- **Color:** {recommended_wine['color']}  
- **Flavor Profile:** {flavor_profile}  
- **Aroma:** {recommended_wine['aromas']}  
- **Sweetness Level:** {sweetness}  
- **Body:** {body}  
- **Price:** {price_range}  
"""
    else:
        prompt = f"""
You are a wine expert. Based on the following attributes, recommend a wine:
- Wine Color: {wine_color}
- Flavor Profile: {flavor_profile}
- Aroma: {aroma}
- Sweetness Level: {sweetness}
- Body: {body}
- Price Range: {price_range}
Provide the best wine recommendation with its name and key details.
"""
        model = genai.GenerativeModel("gemini-pro")
        response = model.generate_content(prompt)
        wine_recommendation = response.text
        wine_name = response.text.split('\n')[0] if response.text else wine_color
    
    return wine_recommendation, wine_name

def update_flavors_aromas(wine_color):
    """
    Updates the dropdown lists for flavor profile and aroma based on the selected wine color.
    """
    valid_flavors = wine_attributes.get(wine_color, {}).get("flavors", [])
    valid_aromas = wine_attributes.get(wine_color, {}).get("aromas", [])
    return gr.update(choices=valid_flavors), gr.update(choices=valid_aromas)

# -------------------------------
# Gradio Interface
# -------------------------------

with gr.Blocks() as demo:
    gr.Markdown("# Wine & Food Pairing Recommendation System")
    
    with gr.Row():
        wine_color_in = gr.Dropdown(wine_colors, label="Wine Color")
        flavor_profile_in = gr.Dropdown([], label="Flavor Profile")
        aroma_in = gr.Dropdown([], label="Aroma Preference")
    
    with gr.Row():
        sweetness_in = gr.Slider(0, 10, step=1, label="Sweetness Level (0 = Dry, 10 = Very Sweet)")
        body_in = gr.Dropdown(body_options, label="Wine Body")
        price_range_in = gr.Dropdown(price_options, label="Price Range")
    
    recommendation_output = gr.Markdown(label="Full Recommendation")
    submit_btn = gr.Button("Get Full Recommendation", variant="primary")
    
    submit_btn.click(get_wine_recommendation, [wine_color_in, flavor_profile_in, aroma_in, sweetness_in, body_in, price_range_in], recommendation_output)
    wine_color_in.change(update_flavors_aromas, inputs=[wine_color_in], outputs=[flavor_profile_in, aroma_in])

demo.launch()


* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\Users\Johnathan\anaconda3\envs\dev1\Lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Johnathan\anaconda3\envs\dev1\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Johnathan\anaconda3\envs\dev1\Lib\site-packages\gradio\blocks.py", line 2052, in process_api
    data = await self.postprocess_data(block_fn, result["prediction"], state)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Johnathan\anaconda3\envs\dev1\Lib\site-packages\gradio\blocks.py", line 1858, in postprocess_data
    prediction_value = block.postprocess(prediction_value)
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Johnathan\anacon